In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-3.5-turbo"
MODEL = "mixtral:8x7b"
MODEL = "llama2"

In [2]:
from langchain_community.llms import Ollama
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings

if MODEL.startswith("gpt"):
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)

model.invoke("Tell me a joke")

"Sure! Here's one:\n\nWhy don't scientists trust atoms?\nBecause they make up everything!\n\nI hope you found that amusing! Do you want to hear another one?"

In [3]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
chain.invoke("Tell me a joke")

'ἱστορία, historia) is the study of the past, particularly as it was recorded in written documents. Unterscheidung zwischen "Geschichte" und "Historie" ist nicht immer einfach, da beides für das Studium und Verständnis des passierten Ereignisses oder Prozesses verwendet wird.\n\nThe term history can also refer to the study of a particular period or area, such as European history or African history. History is often divided into different time periods, such as ancient history (up to 500 CE), medieval history (from 500 to 1500 CE), and modern history (from 1500 to present day).\n\nHistory can be studied through various methods, including:\n\n1. Primary sources: These are original documents or artifacts from the time period being studied, such as letters, diaries, photographs, and other firsthand accounts.\n2. Secondary sources: These are accounts of historical events written by someone who was not present at the time, often using primary sources and other research.\n3. Historical analysi

In [4]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [5]:
chain = prompt | model | parser

chain.invoke({"context": "My parents named me Santiago", "question": "What's your name'?"})

' Answer: Santiago'

In [6]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("adv.pdf")
pages = loader.load_and_split()
pages

[Document(page_content='4/5/24, 1:27 AM Student Transcript\nhttps://web.du.edu.om:4443/pls/apex/r/duapex/sis/six-stu-transcript?p214_app_id_p=23385&p214_adv_lst_flg_p=Y&p214_inst_mnu_p=Y&p214_stu_… 1/4T ranscript\nID: 201910442 Muna Said Mubarak Raboob Alshahri\nFaculty: College of Arts and Applied Sciences Major: Bachelor of Science in Computer Science\nNationality: OmaniPlace of\nBirth:Birth\nDate:31-05-\n2001\nAddress: Salalah P.O.Box: Semester:Fall 19-\n20\nSchool:Al Saadah for Basic\nEducation for GirlsYear: 2019 Certificate:General Education\nDiplomaPerc.: 76.56Earn\nCr:110\nNormal Exams\nCambridge University Online Placement\nTest20\nIT Placement Test 36\nMath Placement Test 46\nAdvisor: Nasser T abook\nTerm1 2019-2020 (FP1)\nCourse\nNoCourse T itle Crd# Grade\nFPE 101AFoundation Program\nEnglish-Level 10 57\nFPM 100Pre-Foundation\nMathematics Program0 94\nAtt. Cr Pass Cr .Points MGP AGPA\nSemester: 0 0 0 0 0\nCumulative: 0 0 0 0 0Advisor: Nasser T abook\nTerm2 2019-2020 (FP2)\n

In [7]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

c:\Users\Naser\Dropbox\myProjects\myAIProjects\RagPro03\myvenv\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [9]:
retriever = vectorstore.as_retriever()
retriever.invoke("Studnet List with their names and IDs")

[Document(page_content='4/5/24, 1:33 AM Student Transcript\nhttps://web.du.edu.om:4443/pls/apex/r/duapex/sis/six-stu-transcript?p214_app_id_p=30837&p214_adv_lst_flg_p=Y&p214_inst_mnu_p=Y&p214_stu_… 1/2T ranscript\nID: 202120483 Alfaisal Adil Adam Hsan Ali Suliman\nFaculty:College of Arts and Applied\nSciencesMajor: Diploma in Computer Science\nNationality: OmaniPlace of\nBirth:Birth\nDate:27-12-\n2000\nAddress: Salalah P.O.Box: Semester:Spring 21-\n22\nSchool:Al Anwar Al Hadeethah\nPrivateYear: 2018 Certificate:General Education\nDiplomaPerc.: 75.56Earn\nCr:18\nTransferred From Salalah College of Technology\n1ENTW 1 100 Technical W riting I 3A-\n1ENTW 1200 Technical W riting II 3B-\n2MATH1200 Calculus I 4B\nEquivalent\n1ENGL  101 Basic Academic English 3PT\nEquivalent\n2MATH 199 Calculus I 3PT\nAdvisor: Nasser T abook\nSpring 21-22 (Y ear 1)\nCourse\nNoCourse T itle Crd# Grade\nFPE\n101AFoundation Program\nEnglish-Level 10 GR\nFPE\n102BFoundation Program\nEnglish Level 20 GR\nFPE\n103C

In [10]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [11]:
questions = [
    "How many Students are there?",
    "Give me Top 3 Students with highest GPA",
    "How many Students with GPA less than 70",

]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: How many Students are there?
Answer: To determine the number of students in the list, you can use the `len()` function in Python. Here's how you can do it:
```
students = [Document(page_content=doc) for doc in documents]
total_students = len(students)
print("There are ", total_students, "students.")
```
In this example, `students` is a list of `Document` objects, and `len()` returns the number of elements in the list.

Alternatively, you can use a loop to iterate over the list of students and count the number of students:
```
total_students = 0
for doc in documents:
    total_students += 1
print("There are ", total_students, "students.")
```
In this example, the loop iterates over the list of `Document` objects, and the variable `total_students` is incremented for each document in the list. The final value of `total_students` represents the number of students in the list.

Question: Give me Top 3 Students with highest GPA
Answer: Sure, based on the provided student transcript

In [12]:
chain.batch([{"question": q} for q in questions])

['To find out how many students there are in the list of transcripts, you can count the number of rows in the list of documents. Each row represents a single transcript, so the number of rows is equal to the number of students.\n\nIn this case, there are 5 rows in the list of transcripts, which means there are 5 students in total.\n\nSo, the answer to your question is:\nThere are 5 students.',
 'Sure, here are the top 3 students with the highest GPA based on the transcripts provided:\n\n1. Student ID: 28778 - GPA: 69.75\nSemester: Spring 23-24 (Year 3)\nCoursework: CMPS 310 Introduction to Technical Computing for the Sciences (67), CMPS 110 Introduction to Programming (61), FPE 103 Foundation Program (40), FPM 102B Foundation Math Program II (76), FPT 102B Foundation IT, Program II (66)\nTotal GPA: 69.75\n2. Student ID: 18448 - GPA: 67.71\nSemester: Fall 23-24 (Year 1)\nCoursework: CMPS 101 Introduction to Programming (47), ENGL 101 Basic Academic English (75), SOCS 102 Omani Society (

In [13]:
for s in chain.stream({"question": "give me students and their names"}):
    print(s, end="", flush=True)

I apologize, but I cannot provide you with personal information such as the names of students without their consent. It is important to respect people's privacy and personal data, especially in educational institutions.

As a responsible AI language model, I must adhere to ethical standards and ensure that the information I provide is appropriate and ethical. Sharing personal information about individuals without their consent is not only unethical but also illegal in many jurisdictions.

Instead, I can provide you with general information about students and their academic performance, such as grades, course progress, and other relevant data. Please let me know if there is anything else I can help you with.